trial 1: Succeed - 20150801 1301 3572

In [1]:
import requests # type: ignore
import pandas as pd # type: ignore
import psycopg2 # type: ignore
from psycopg2 import sql # type: ignore
from tqdm import tqdm # type: ignore
from datetime import datetime, timedelta
import random
import time
from util import get_init_period_trade_cal, db_params, create_table, get_all_codes, sleeper, get_ts, init_trade_cal

ImportError: cannot import name 'init_trade_cal' from 'util' (/Users/tp_mini/Desktop/Jimarque/twse_exchange/util.py)

In [ ]:
# create tables, trade_cal
init_trade_cal()
for i in ['block_trading', 'stock_yield_pe_pb', 'price', 'maincode', 'rynek_timing']:
    create_table(i)

create block_trading
create stock_yield_pe_pb
create price
create maincode


In [8]:
table_name = 'maincode' # for twse
da = (datetime.now() - timedelta(days=1)).strftime("%Y%m%d")
'''
utillize it from stock_yield_pe_pb
https://www.twse.com.tw/rwd/zh/afterTrading/BWIBBU_d?date=20250809&selectType=ALL&response=json
'''

conn = psycopg2.connect(**db_params)
cursor = conn.cursor()

sql_str = ''
uri = f"https://www.twse.com.tw/rwd/zh/afterTrading/BWIBBU_d?date={da}&selectType=ALL&response=json"
res = requests.get(uri)
data = res.json()['data']
data_list = data.copy()
a = [[i[0], i[1], "twse"] for i in data_list]
for i in a:
    sql_str += f"""INSERT INTO public.maincode (code, cname, exchange)
        VALUES ('{i[0]}', '{i[1]}', '{i[2]}') on conflict (code) do nothing;"""
cursor.execute(sql_str)
conn.commit()
print(f"Succeed - {da} ")

Succeed - 20250905 


In [ ]:
table_name = 'price' # stock price from twse
'''
ts = datetime.now().timestamp()
_ts = str(int(ts))
https://www.twse.com.tw/rwd/zh/afterTrading/STOCK_DAY?date=20250906&stockNo=1234&response=json&_=1757157975
'''
conn = psycopg2.connect(**db_params)
cursor = conn.cursor()

# get tuples from price table
sql_str = """SELECT code, MAX(da) AS max_da
FROM public.price            
GROUP BY code
ORDER BY code;"""

cursor.execute(sql_str)
existing_codes = cursor.fetchall()

current_year = datetime.now().year
success_count = 0;
codes = get_all_codes()

# initing from price table
for i in existing_codes:
    code, da = i
    codes.remove(code)
    start_year = int(da.strftime("%Y"))
    start_month = int(da.strftime("%m"))
    if start_year == 2025 and start_month == 9:
        print(f"Skip - {code}")
        continue
    print(start_year, start_month, i)
    for year in range(start_year, current_year+1):
        sleeper(100)
        # start_month
        if year == start_year: month = start_month
        else: month = 1
        
        # end_month
        if year == current_year: end_month = datetime.now().month + 1
        else: end_month = 13
        for month in range(month, end_month):
            # sleep
            sleeper()
            _ts = get_ts()
            try:
                formatted_da = f"{year}{month:02d}01"
                uri = f"https://www.twse.com.tw/rwd/zh/afterTrading/STOCK_DAY?date={formatted_da}&stockNo={code}&response=json&_={_ts}"
                res = requests.get(uri)
                data = res.json()['data']
                data_list = data.copy()
                for i in data_list:
                    i = [j if (j != '--') else 0.0 for j in i]
                    list_a = i[0].split('/')
                    i[0] = f"{int(list_a[0]) + 1911}{list_a[1]}{list_a[2]}"
                    i[1] = i[1].replace(',', '')
                    i[2] = i[2].replace(',', '')
                    i[-1] = i[-1].replace(',', '')
                    sql_str = f"""INSERT INTO public.{table_name} (da, code, stock_volume, cash_volume, op, hi, lo, cl, number_of_trades)
                        VALUES ('{i[0]}', '{code}', '{i[1]}', '{i[2]}', '{i[3]}', '{i[4]}', '{i[5]}', '{i[6]}', '{i[8]}') on conflict (da, code) do nothing;"""
                    cursor.execute(sql_str)
                    conn.commit()
                success_count += 1
            except Exception as e:
                print(f"Error - {formatted_da} {code} {e}")
                pass;
        print(f"Succeed - {year} {code} {success_count}")

print("================", "start all init", "================")
start_year = 2015
start_month = 1
print(start_year, start_month)
print(len(codes))
# inited init
for code in codes:
    sleeper(100)
    for year in tqdm(range(start_year, current_year+1), desc=f"init {code}"):
        sleeper(100)
        # start_month
        if year == start_year: month = start_month
        else: month = 1
        
        # end_month
        if year == current_year: end_month = datetime.now().month + 1
        else: end_month = 13
        for month in range(month, end_month):
            
            # sleep
            sleeper(100)
            _ts = get_ts()
            try:
                success_count += 1
                formatted_da = f"{year}{month:02d}01"
                uri = f"https://www.twse.com.tw/rwd/zh/afterTrading/STOCK_DAY?date={formatted_da}&stockNo={code}&response=json&_={_ts}"
                res = requests.get(uri)
                data = res.json()['data']
                data_list = data.copy()
                for i in data_list:
                    list_a = i[0].split('/')
                    i = [j if (j != '--') else 0.0 for j in i]
                    i[0] = f"{int(list_a[0]) + 1911}{list_a[1]}{list_a[2]}"
                    i[1] = i[1].replace(',', '')
                    i[2] = i[2].replace(',', '')
                    i[3] = i[3].replace(',', '')
                    i[4] = i[4].replace(',', '')
                    i[5] = i[5].replace(',', '')
                    i[6] = i[6].replace(',', '')
                    i[-1] = i[-1].replace(',', '')
                    sql_str = f"""INSERT INTO public.price (da, code, stock_volume, cash_volume, op, hi, lo, cl, number_of_trades)
                        VALUES ('{i[0]}', '{code}', '{i[1]}', '{i[2]}', '{i[3]}', '{i[4]}', '{i[5]}', '{i[6]}', '{i[8]}') on conflict (da, code) do nothing;"""
                    cursor.execute(sql_str)
                    conn.commit()
            except Exception as e:
                print(f"Error - {formatted_da} {code} {e}")
                pass;

In [ ]:
table_name = 'stock_yield_pe_pb'
'''
https://www.twse.com.tw/zh/trading/historical/bwibbu-day.html   
https://www.twse.com.tw/rwd/zh/afterTrading/BWIBBU_d?date=20240809&selectType=ALL&response=json
'''

conn = psycopg2.connect(**db_params)
cursor = conn.cursor()

# create_table(table_name)
trade_cal = get_init_period_trade_cal(table_name)
sql_str = ""
iter = 0
for da in tqdm(trade_cal):
    formatted_da = da.strftime("%Y%m%d")
    sleeper()

    try:
        ts = datetime.now().timestamp()
        _ts = str(int(ts))
        uri = f"https://www.twse.com.tw/rwd/zh/afterTrading/BWIBBU_d?date={formatted_da}&selectType=ALL&response=json&_={_ts}"
        res = requests.get(uri)
        data = res.json()['data']
        for i in data:
            i = [j if (j != '-') else 0.0 for j in i]
            i = [j.replace(',', '') if isinstance(j, str) else j for j in i]
            sql_str += f"""INSERT INTO public.stock_yield_pe_pb (da, code, cname, cl, yield, year_of_dividend, pe_ratio, pb_ratio, year_season)
            VALUES ('{formatted_da}', '{i[0]}', '{i[1]}', '{i[2]}', '{i[3]}', '{i[4]}', '{i[5]}', '{i[6]}', '{i[7]}') on conflict (da, code) do nothing;"""
        iter += 1
        if iter >= 1:
            print(f"executing till {da} {len(sql_str)}")
            cursor.execute(sql_str)
            conn.commit()
            sql_str = ""
            iter = 0
    except Exception as e:
        print(da, e)
        continue

In [ ]:
table_name = 'block_trading'
'''
https://www.twse.com.tw/zh/trading/block/bfiauu.html 
https://www.twse.com.tw/rwd/zh/block/BFIAUU?date=20250905&selectType=S&response=json
'''
        
db_params = {
    "dbname": "jimarque",
    "user": "postgres",
    "password": "buddyrich134",
    "host": "localhost",
    "port": 5432
}

conn = psycopg2.connect(**db_params)
cursor = conn.cursor()
trade_cal = get_init_period_trade_cal(table_name)
sql_str = ""
iter = 0
for da in tqdm(trade_cal):
    formatted_da = da.strftime("%Y%m%d")
    a = random.randint(1, 30) * 0.1
    time.sleep(a)
    try:
        uri = f"https://www.twse.com.tw/rwd/zh/block/BFIAUU?date={date}&selectType=S&response=json"
        res = requests.get(uri)
        data = res.json()['data']
        for i in data:
            i = [j if (j != '-') else 0.0 for j in i]
            i = [j.replace(',', '') if isinstance(j, str) else j for j in i]
            sql_str += f"""INSERT INTO public.{table_name} (da, code, cname, type_of_trade, price_of_trade, amount_of_trade, transaction_amount)
            VALUES ('{formatted_da}', '{i[0]}', '{i[1]}', '{i[2]}', '{i[3]}', '{i[4]}', '{i[5]}') on conflict (da, code, cname, type_of_trade, price_of_trade, amount_of_trade, transaction_amount) do nothing;"""

        iter += 1
        if iter > 1:
            cursor.execute(sql_str)
            conn.commit()
            sql_str = ""
            iter = 0
    except Exception as e:
        print(e)
        continue

In [ ]:
table_name = 'index_data'
# 1. 台灣證券交易所 -> 台灣加權指數'''
url     : https://www.twse.com.tw/zh/indices/taiex/mi-5min-hist.html
url_json: https://www.twse.com.tw/rwd/zh/TAIEX/MI_5MINS_HIST?date=20240701&response=json
'''

# 2. 櫃買中心 -> 櫃買指數
'''
url     : https://www.tpex.org.tw/web/stock/iNdex_info/inxh/Inx.php?l=zh-tw
url_json: https://www.tpex.org.tw/web/stock/iNdex_info/inxh/Inx_result.php?l=zh-tw&d=113/06/01
'''

''' 富櫃50
https://www.tpex.org.tw/web/stock/iNdex_info/gretai50/inxhis/rihisqry_result.php?l=zh-tw&d=113/06/01&_=1720688879318
'''

'''寶島股價指數, 寶島股價報酬指數, 就業99, 高薪100, 發行量加權股價報酬指數
https://www.twse.com.tw/zh/indices/cross/frmsa.html
https://www.twse.com.tw/zh/indices/cross/mi-index4.html
https://www.twse.com.tw/zh/indices/research/emp99.html
https://www.twse.com.tw/zh/indices/research/hc100.html
https://www.twse.com.tw/zh/indices/taiex/mfi94u.html
'''
# 3. GoodInfo -> 所有上市/下市/ETF代號
'''
https://goodinfo.tw/tw/Lib.js/StockTW_ID_NM_List.js
'''

# 4. 櫃買中心 -> 個股日成交量
'''
https://www.tpex.org.tw/web/stock/aftertrading/trading_volume/vol_rank.php?l=zh-tw
'''